In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import unicodedata
import re
from IPython.display import FileLink


In [156]:
ratings_path = r'C:\Users\Βαγγέλης\Desktop\Σχολή\4o Έτος\7ο Εξάμηνο\Ειδικό Θέμα με Τουσίδου (ομαδικό με Σταμάτη)\Datasets\Dataset_5\Ratings.csv'
users_path = r'C:\Users\Βαγγέλης\Desktop\Σχολή\4o Έτος\7ο Εξάμηνο\Ειδικό Θέμα με Τουσίδου (ομαδικό με Σταμάτη)\Datasets\Dataset_5\Users.csv'
books_path = r'C:\Users\Βαγγέλης\Desktop\Σχολή\4o Έτος\7ο Εξάμηνο\Ειδικό Θέμα με Τουσίδου (ομαδικό με Σταμάτη)\Datasets\Dataset_5\Books.csv'

In [157]:
#fortwnoume ta datasets
ratings_df = pd.read_csv(ratings_path, on_bad_lines='skip', encoding='latin1')
users_df = pd.read_csv(users_path, on_bad_lines='skip', encoding='latin1')
books_df = pd.read_csv(books_path, on_bad_lines='skip', low_memory=False, encoding='latin1')

In [158]:
print(ratings_df.columns)
print(users_df.columns)
print(books_df.columns)

Index(['User-ID', 'ISBN', 'Book-Rating'], dtype='object')
Index(['User-ID', 'Location', 'Age'], dtype='object')
Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')


In [159]:
books_df.drop(columns=['Image-URL-S', 'Image-URL-M', 'Image-URL-L'], inplace=True)

In [160]:
books_df.columns = ['ISBN', 'Title', 'Author', 'Publication_Date', 'Publisher']
ratings_df.columns = ['User_ID', 'ISBN', 'Rating']
users_df.columns = ['User_ID', 'Location', 'Age']

In [161]:
print(ratings_df.columns)
print(users_df.columns)
print(books_df.columns)

Index(['User_ID', 'ISBN', 'Rating'], dtype='object')
Index(['User_ID', 'Location', 'Age'], dtype='object')
Index(['ISBN', 'Title', 'Author', 'Publication_Date', 'Publisher'], dtype='object')


In [162]:
ratings_df.dropna(subset=['ISBN', 'User_ID'], inplace=True)
books_df.dropna(subset=['ISBN', 'Title'], inplace=True)

In [163]:
books_df = books_df.drop_duplicates(subset=['Title', 'ISBN'], keep='first').reset_index(drop=True)

>Books csv

In [164]:
books_df['Publication_Date'] = pd.to_numeric(books_df['Publication_Date'], errors='coerce')

current_year = pd.Timestamp.now().year
books_df = books_df[(books_df['Publication_Date'] >= 1800) & (books_df['Publication_Date'] <= current_year)]

books_df['Publication_Date'] = books_df['Publication_Date'].astype(int)

In [165]:
def remove_accents(text):
    return ''.join(c for c in unicodedata.normalize('NFKD', text) if not unicodedata.combining(c))

def remove_special_characters(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('ascii')

books_df['Title'] = books_df['Title'].apply(lambda x: remove_special_characters(remove_accents(x)).lower().strip())

special_characters = books_df['Title'].str.contains(r'[^\x00-\x7F]', regex=True)
print("Rows with special characters in 'Title' after normalization:", books_df[special_characters])

Rows with special characters in 'Title' after normalization: Empty DataFrame
Columns: [ISBN, Title, Author, Publication_Date, Publisher]
Index: []


In [166]:
books_df['Author'] = books_df['Author'].fillna('Unknown').astype(str).str.lower().str.strip()

In [167]:
books_df['Publisher'] = books_df['Publisher'].fillna('Unknown').astype(str).str.lower().str.strip()

label_encoder = LabelEncoder()

books_df['Publisher_Encoded'] = label_encoder.fit_transform(books_df['Publisher'])

In [168]:
print(ratings_df.columns)
print(users_df.columns)
print(books_df.columns)

Index(['User_ID', 'ISBN', 'Rating'], dtype='object')
Index(['User_ID', 'Location', 'Age'], dtype='object')
Index(['ISBN', 'Title', 'Author', 'Publication_Date', 'Publisher',
       'Publisher_Encoded'],
      dtype='object')


>Users csv

In [169]:
users_df['Age'] = users_df['Age'].astype(str)

users_df['Age'] = users_df['Age'].replace(['nan', 'NaN', 'None', ''], 'Unknown')

users_df['Age'] = users_df['Age'].fillna("Unknown").apply(lambda x: int(x) if isinstance(x, float) else x)

In [170]:
users_df['Age'] = pd.to_numeric(users_df['Age'], errors='coerce')

users_df.loc[(users_df['Age'] < 18) | (users_df['Age'] > 100), 'Age'] = np.nan

users_df['Age'] = users_df['Age'].fillna("Unknown")

users_df.reset_index(drop=True, inplace=True)

>Ratings csv

In [171]:
ratings_df = ratings_df[ratings_df['Rating'] != 0]

In [172]:
user_avg_ratings = ratings_df.groupby('User_ID')['Rating'].transform('mean').round(1)

ratings_df['User_Avg_Rating'] = user_avg_ratings

print(ratings_df.head())

   User_ID        ISBN  Rating  User_Avg_Rating
1   276726  0155061224       5              5.0
3   276729  052165615X       3              4.5
4   276729  0521795028       6              4.5
6   276736  3257224281       8              8.0
7   276737  0600570967       6              6.0


In [173]:
books_df.to_csv('cleaned_books.csv', index=False, encoding='utf-8')
users_df.to_csv('cleaned_users.csv', index=False, encoding='utf-8')
ratings_df.to_csv('cleaned_ratings.csv', index=False, encoding='utf-8')
user_ratings_df = pd.merge(ratings_df, users_df, on="User_ID", how="inner")
user_ratings_df.to_csv("cleaned_user_ratings.csv", index=False)